In [1]:
# counting bloom filter class

import math
# for hashing 
import hashlib 

class counting_bloom_filter: 
    
    def __init__(self, input_size, fpr): 
        # expected input size 
        self.input_size = input_size 
        # chosen false positive rate 
        self.fpr = fpr 
        
        # get array size 
        self.array_size = -1*round((self.input_size*math.log(self.fpr))/(math.log(2))**2)
        # create array of zeroes
        self.array = [0]*self.array_size
        
        # get optimal # of hashing functions 
        self.hash_functions = round((self.array_size/self.input_size)*math.log(2))
        # make sure there is at least one hashing function
        if (self.hash_functions < 1):
            self.hash_functions = 1
        # set limit to six hashing functions commonly contained in the hashlib library
        elif (self.hash_functions > 6):
            self.hash_functions = 6
        
    # method for inserting elements
    def insert(self, element):
        # get hash keys 
        keys = self.hash(element)
        # reset current key
        key = 0
        # iterate through each key
        for i in range(self.hash_functions):
            key = int(keys[i]%self.array_size)
            # increment counter
            self.array[key] += 1
            
    # method for finding elements
    def find(self, element):
        # initialize boolean variable to see if element exists
        is_found = True
        keys = self.hash(element)
        key = 0
        for i in range(self.hash_functions):
            key = int(keys[i]%self.array_size)
            # if the element exists
            if (not self.array[key] > 0):
                # set is_found to True
                is_found = False
        # return true/false
        return is_found
        
    
    # method for removing elements 
    def remove(self, element):
        keys = self.hash(element)
        key = 0
        # check if element exists
        if (self.find(element)):
            for i in range(self.hash_functions):
                key = int(keys[i]%self.array_size)
                # decrement counter
                self.array[key] -= 1
        # if it doesn't
        else:
            print('That name does not exist.')
            
    
    # method for defining hash functions 
    def hash(self, element):
        # intialize list for keys
        keys = []
        
        # hash functions 1-6
        h_1 = hashlib.sha1(element.encode())
        h_2 = hashlib.sha224(element.encode())
        h_3 = hashlib.sha256(element.encode())
        h_4 = hashlib.sha384(element.encode())
        h_5 = hashlib.sha512(element.encode())
        h_6 = hashlib.md5(element.encode())
        
        # append keys to list
        keys.append(int(h_1.hexdigest(),16))
        keys.append(int(h_2.hexdigest(),16))
        keys.append(int(h_3.hexdigest(),16))
        keys.append(int(h_4.hexdigest(),16))
        keys.append(int(h_5.hexdigest(),16))
        keys.append(int(h_6.hexdigest(),16))
        
        # return list of keys
        return keys 

In [2]:
# process names -> make lowercase, remove spaces for more accurate comparisons

def process_names(name): 
    # check if input is a string 
    if (type(name) != str):
        raise TypeError('Input must be a string. Please try again.')
        
    # convert to lowercase, remove any spaces before/after name
    processed_name = name.lower().strip()
    
    # return processed name
    return processed_name

In [3]:
# extract names from .csv file 

# import csv libary 
import csv 
# for returning lower-case letters only 
from string import ascii_lowercase

def extract_names(filename): 
    
    # open file
    f = open(filename)
    
    with f as file:
        # read file, using comma as delimiter
        read_file = csv.reader(file, delimiter=',')
        # count lines 
        l_count = 0
        
        # initialize empty list to hold names
        names = {}
        for i in ascii_lowercase:
            names[i] = []
        cbf = counting_bloom_filter(400, 0.001)

        # get name from each row
        for row in read_file:
            # if not first line 
            if l_count != 0:
                # process name & add to counting bloom filter
                names[process_names(row[0][0])].append(process_names(row[0]))
                cbf.insert(process_names(row[0]))
            # increment counter 
            l_count += 1
        
    # return list of names 
    return(names, cbf)

In [4]:
# longest common subsequence functions 

import numpy as np

# returns length of longest common subsequence between two strings
def longest_common_subsequence(x,y): 
    # check if lengths are zero
    if (len(x)==0) or (len(y)==0):
        return None 
    # get lengths of lists 
    i, j = len(x), len(y)
    c, b = lcs_length(x,y)
    # initialize list to hold lcs
    lcs = []
    # return length of lcs 
    return(traverse(b,x,i,j,lcs)[1])

# computes length of lcs
def lcs_length(x,y):
    # length of lists
    m, n = len(x), len(y)
    # table containing lengths of lcs of x[:i] and y[:j]
    c = [[0 for i in range(n+1)] for j in range(m+1)]
    # table containing directions for lcs reconstruction
    b = [[0 for i in range(n+1)] for j in range(m+1)]
   
    # iterate through list x
    for i in range(1, m+1):
        # iterate through list y
        for j in range(1, n+1):
            
            # if element from x is same as element from y 
            if x[i-1] == y[j-1]:
                # increment value by 1
                c[i][j] = c[i-1][j-1]+1
                # set direction to northwest
                b[i][j] = 'NW'
            # check which is greater 
            elif c[i-1][j] >= c[i][j-1]:
                c[i][j] = c[i-1][j]
                # set direction to north
                b[i][j] = 'N'
            # else:
            else:
                c[i][j] = c[i][j-1]
                # set direction to west
                b[i][j] = 'W'
                
    # return
    return c, b

# find lcs using directions
def traverse(b,x,i,j,lst): 
    # check if indices = 0
    if i==0 or j==0:
        # end recursion
        return 
    
    # if direction is NW 
    if b[i][j] == 'NW':
        # recursively call function 
        traverse(b,x,i-1,j-1,lst)
        # append element to list 
        lst.append(x[i-1])
        
    # if North
    elif b[i][j] == 'N':
        traverse(b,x,i-1,j,lst)
        
    # if West
    else:
        traverse(b,x,i,j-1,lst)
        
    # return list & its length
    return[lst,len(lst)]

In [5]:
# test lcs 
assert(longest_common_subsequence('a','b')==0) 
assert(longest_common_subsequence('abcdefg','ajklcdef')==5)
assert(longest_common_subsequence('ABC','abc')==0)

In [6]:
# check if name is spelled correctly 
def check_names(input_name, cbf):
    # process name & see if it can be found in cbf
    return cbf.find(process_names(input_name))

In [7]:
# suggest possible names 
def suggest_names(input_name, user_names):
    
    # initialize variable to hold longest LCS 
    max_lcs = -float('inf')
    # initalize variable for best suggestion
    closest_name = ''
    # intialize boolean variable for whether name is found
    is_found = False 
    
    # iterate through every name that starts with the same first letter as inputted name
    for name in user_names[input_name[0]]:
        # get lcs
        lcs = longest_common_subsequence(input_name, name)
        # check if length of lcs is > length of possible name by 60% 
        if lcs > math.floor(0.60*(len(name))):
            # check if length of input_name is between 60% to 140% of the length of the possible name
            if (lcs > max_lcs) and (math.floor(0.6*len(input_name))<=len(name) and 
                                    math.floor(1.4*len(input_name))>=len(name)):
                # update max_lcs, closest_name, & is_found
                max_lcs = lcs
                closest_name = name
                is_found = True
    
    # if name is not yet found 
    if is_found == False: 
        # iterate through all keys
        for letter in user_names:
            # iterate through all names 
            for name in user_names[letter]:
                # get lcs
                lcs = longest_common_subsequence(input_name, name)
                # check conditions 
                if lcs > math.floor(0.6*len(name)):
                    if (lcs > max_lcs) and (math.floor(0.6*len(input_name))<=len(name) and 
                                            (math.floor(1.4*len(input_name))>=len(name))):
                        # update
                        max_lcs = lcs
                        closest_name = name
                        is_found = True
    
    # if still no name found 
    if closest_name == '':
        # return None
        return None 
    
    
    # return name, capitalized
    return(closest_name.capitalize())

In [8]:
# main function
def name_checker (input_name, filename):
    # if input is empty 
    if input_name == '':
        return 
    # process input for more accurate comparison
    input_name = process_names(input_name)
    # extract names from .csv file
    names = extract_names(filename)
    # initialize variables for cbf & dictionary 
    cbf = names[1]
    names_dict = names[0]
    # check if name is spelled correctly
    if (check_names(input_name,cbf)):
        return True 
    # if not, suggest closest name 
    else:
        return suggest_names(input_name, names_dict)
    suggest_names(input_name, names_dict)
    

In [9]:
# test cases

# normal cases w/ small error
assert(name_checker('Audey', 'usernames.csv')=='Audrey')
assert(name_checker('Li', 'usernames.csv')=='L')
# normal case where input is correct
assert(name_checker('Hai', 'usernames.csv')==True)
# normal case where input does not exist
assert(name_checker('Apple', 'usernames.csv')==None)

# phonetically similar
assert(name_checker('Alice', 'usernames.csv')=='Allyce')
assert(name_checker('Cornelia', 'usernames.csv')=='Kornelija')

# edge case - no input 
assert(name_checker('', 'usernames.csv')==None)
# edge case - spaces
assert(name_checker('B ry an', 'usernames.csv')=='Brian')

In [10]:
# create GUI for easier usage

from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets

# create text box for user to enter name
text_box = widgets.Text(
    placeholder='Enter a name',
    disabled=False,
)

# create button to check name
btn = widgets.Button(description='check name', button_style='info')
output = widgets.Output()

# what to do when button is clicked
def btn_eventhandler(obj):
    # call name_checker function
    check = name_checker(text_box.value, 'usernames.csv')
    # spelled correctly
    if check == True:
        print('This name is spelled correctly!')
    # suggest name
    elif check != None: 
        print('Closest match:',check)
    # try again
    else:
        print('We were unable to find a suitable match. Please try again.')

# when button clicked
btn.on_click(btn_eventhandler)

# format horizontally
layout = widgets.HBox([text_box, btn])
# display
display(layout)